# Machine Learning with PyTorch

## Natural Language Processing with AllenNLP

<font size="+1">What is AllenNLP?</font>
<a href="AllenNLP_0.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">What is SpaCy?</font>
<a href="AllenNLP_1.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">Hight Level Interfaces to NLP using PyTorch</font>
<a href="AllenNLP_2.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">Sentiment Analysis</font>
<a href="AllenNLP_3.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font> 
<a href="AllenNLP_4.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font>
<a href="AllenNLP_5.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font>
<a href="AllenNLP_6.ipynb"><img src="img/open-notebook.png" align="right"/></a>

## Credit

The material in this section is borrowed from Masato Hagiwara's [Training a Sentiment Analyzer using AllenNLP (in less than 100 lines of Python code)](http://www.realworldnlpbook.com/blog/training-sentiment-analyzer-using-allennlp.html).  I have made some minor changes to the code and provided my own commentary; I recommend all of his blog posts and other writing highly.

The dataset used in this example by Hagiwara, and hence by me, are provided by Stanford University's [Sentiment Analysis](https://nlp.stanford.edu/sentiment/index.html) research page.  This dataset was presented in the paper _Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank_ by Richard Socher, Alex Perelygin, Jean Wu, Jason Chuang, Christopher Manning, Andrew Ng and Christopher Potts.  The dataset is provided with this repository for convenience.

In [2]:
# AllenNLP comes with a reader for this format 
from allennlp.data.dataset_readers import stanford_sentiment_tree_bank
SSTBDR = stanford_sentiment_tree_bank.StanfordSentimentTreeBankDatasetReader
reader = SSTBDR()

train_dataset = reader.read('data/stanfordSentimentTreebank/trees/train.txt')
dev_dataset = reader.read('data/stanfordSentimentTreebank/trees/dev.txt')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


0it [00:00, ?it/s]

FileNotFoundError: file data/stanfordSentimentTreebank/trees/train.txt not found

## Next Lesson

**Enhancing an Image Classifier**: Libraries built on top of PyTorch offer very powerful tools for natural language processing.  Next we will return to image classification, and build on pretrained models as a demonstration of re-use neural networks.

<a href="ImageClassifier.ipynb"><img src="img/open-notebook.png" align="left"/></a>